# Geração da Tabela Base de Origem

In [1]:
# Esse processo preciso rodar diariamente e localmente

import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
import streamlit as st

def get_fii_table():
    url = "https://www.fundsexplorer.com.br/ranking"

    options = Options()
    # options.add_argument("--headless=new")
    # options.add_argument("--no-sandbox")
    # options.add_argument("--disable-dev-shm-usage")

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    wait = WebDriverWait(driver, 20)

    try:
        driver.get(url)
        time.sleep(2)  # espera inicial

        # --- 0) Tenta fechar/aceitar o cookie banner por vários caminhos ---
        cookie_selectors = [
            'button[data-test="accept"]',
            'button#hs-eu-confirmation-button',             # exemplos comuns
            'button[aria-label*="aceitar"]',
            'button:contains("Aceitar")',                   # fallback textual (pode não funcionar no CSS)
            'div#hs-en-cookie-confirmation-buttons-area button',
            'button:contains("Aceitar todos")'
        ]
        # tentar por texto também (mais confiável)
        texts_to_try = ["Aceitar todos", "Aceitar", "OK", "Fechar"]

        # 1) tentar clique por seletores diretos
        for sel in cookie_selectors:
            try:
                els = driver.find_elements(By.CSS_SELECTOR, sel)
                if els:
                    for e in els:
                        try:
                            driver.execute_script("arguments[0].scrollIntoView(true);", e)
                            driver.execute_script("arguments[0].click();", e)
                            time.sleep(0.4)
                        except Exception:
                            pass
                    # se algum botão foi clicado, pausa e tenta prosseguir
                    time.sleep(0.6)
            except Exception:
                pass

        # 2) tentar clicar por texto (procura por botões/links)
        for txt in texts_to_try:
            try:
                candidates = driver.find_elements(By.XPATH, f"//button[contains(translate(., 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), '{txt.lower()}') or //a[contains(translate(., 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), '{txt.lower()}')]]")
                if candidates:
                    for c in candidates:
                        try:
                            driver.execute_script("arguments[0].scrollIntoView(true);", c)
                            driver.execute_script("arguments[0].click();", c)
                            time.sleep(0.4)
                        except Exception:
                            pass
                    time.sleep(0.6)
            except Exception:
                pass

        # 3) se ainda existir o elemento de cookie conhecido, remove via JS (forçado)
        try:
            driver.execute_script("""
                var el = document.getElementById('hs-en-cookie-confirmation-buttons-area');
                if (el) { el.remove(); }
                var el2 = document.querySelector('[id^=hs-en-cookie]'); if(el2) el2.remove();
                var overlays = document.querySelectorAll('.cookie, .cookies, .hs-cookie-banner'); 
                overlays.forEach(e => e.remove());
            """)
            time.sleep(0.4)
        except Exception:
            pass

        # --- 4) abrir o menu de colunas (scroll + click via JS para evitar intercept) ---
        botao_colunas = wait.until(EC.presence_of_element_located((By.ID, "colunas-ranking__select-button")))
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", botao_colunas)
        try:
            # preferencial: click via JS para evitar intercept
            driver.execute_script("arguments[0].click();", botao_colunas)
        except Exception:
            # fallback: webdriver click
            botao_colunas.click()
        time.sleep(0.6)

        # --- 5) clicar em "todos" (label) ---
        label_todos = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'label[for="colunas-ranking__todos"]')))
        driver.execute_script("arguments[0].scrollIntoView(true);", label_todos)
        try:
            driver.execute_script("arguments[0].click();", label_todos)
        except Exception:
            label_todos.click()
        time.sleep(1.0)  # deixa o JS atualizar a tabela

        # --- 6) esperar a tabela estar populada ---
        def tabela_populada(driver):
            try:
                rows = driver.find_elements(By.CSS_SELECTOR, ".default-fiis-table__container__table tbody tr")
                # contar linhas não-vazias
                count = 0
                for r in rows:
                    tds = r.find_elements(By.TAG_NAME, "td")
                    if any(td.text.strip() for td in tds):
                        count += 1
                return count > 5  # ajuste se precisar
            except:
                return False

        wait.until(tabela_populada)

        # --- 7) pegar HTML da tabela ---
        tabela = driver.find_element(By.CSS_SELECTOR, ".default-fiis-table__container__table")
        html = tabela.get_attribute("outerHTML")
        df = pd.read_html(html)[0]
        return df

    finally:
        driver.quit()

#Call the function to test
df_fiis = get_fii_table()

def carregar_dados(df=df_fiis):

    df = df.dropna(subset=[
        'P/VP',
        'DY (3M) Acumulado',
        'DY (6M) Acumulado',
        'DY (12M) Acumulado',
        'Liquidez Diária (R$)',
        'Patrimônio Líquido',
        'Num. Cotistas',
        'Preço Atual (R$)'
    ])

    df['P/VP'] = df['P/VP'] / 100

    for col in ['DY (3M) Acumulado', 'DY (6M) Acumulado', 'DY (12M) Acumulado']:
        df[col] = (
            df[col].astype(str)
            .str.replace('%', '', regex=False)
            .str.replace('.', '', regex=False)
            .str.replace(',', '.', regex=False)
            .astype(float)
        )

    df['Liquidez Diária (R$)'] = (
        df['Liquidez Diária (R$)']
        .astype(str).str.replace('.', '', regex=False)
        .str.replace(',', '.', regex=False)
        .astype(float) / 1_000_000
    )

    df['Patrimônio Líquido'] = (
        df['Patrimônio Líquido']
        .astype(str).str.replace('.', '', regex=False)
        .str.replace(',', '.', regex=False)
        .astype(float) / 1_000_000
    )

    df['Num. Cotistas'] = (
        df['Num. Cotistas']
        .astype(str).str.replace('.', '', regex=False)
        .str.replace(',', '.', regex=False)
        .astype(float) / 1_000
    )

    df['Preço Atual (R$)'] = (
        df['Preço Atual (R$)']
        .astype(str).str.replace('.', '', regex=False)
        .str.replace(',', '.', regex=False)
        .astype(float) / 100
    )

    df['Último Dividendo'] = (
        df['Último Dividendo']
        .astype(str).str.replace('.', '', regex=False)
        .str.replace(',', '.', regex=False)
        .astype(float) / 100
    )

    df.rename(columns={
        'Liquidez Diária (R$)': 'Liquidez Diária (milhões R$)',
        'Patrimônio Líquido': 'Patrimônio Líquido (milhões R$)',
        'Num. Cotistas': 'Num. Cotistas (milhares)'
    }, inplace=True)

    return df

df_fiis = carregar_dados(df_fiis)
today_str = pd.Timestamp.today().strftime('%Y-%m-%d')

ajustes_manuais_segmento = pd.read_csv('ajustes_manuais_segmento.csv')

df_fiis = df_fiis.merge(ajustes_manuais_segmento, on='Fundos', how='left', suffixes=('', '_novo'))
df_fiis['Setor'] = df_fiis['Setor_novo'].combine_first(df_fiis['Setor'])
df_fiis.drop(columns=['Setor_novo'], inplace=True)

# Salvando os Top 10 FIIs Descontados com Qualidade
def filtrar_fiis_descontados_com_qualidade(df):
    filtros = (
        (df["P/VP"] >= 0.85) &
        (df["P/VP"] < 1.0) &
        (df["DY (3M) Acumulado"] >= 3) &
        (df["DY (6M) Acumulado"] >= 6) &
        (df["DY (12M) Acumulado"] >= 12) &
        (df["Liquidez Diária (milhões R$)"] >= 1) &
        (df["Patrimônio Líquido (milhões R$)"] >= 500) &
        (df["Num. Cotistas (milhares)"] >= 10)
    )
    return df[filtros].copy()

df_filtrados = filtrar_fiis_descontados_com_qualidade(df_fiis)

df_top10 = (
    df_filtrados
    .sort_values("DY (12M) Acumulado", ascending=False)
    .head(10)
) 

C:\Users\danie\AppData\Local\Temp\ipykernel_5428\522615446.py:128: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(html)[0]
C:\Users\danie\AppData\Local\Temp\ipykernel_5428\522615446.py:150: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['P/VP'] = df['P/VP'] / 100
C:\Users\danie\AppData\Local\Temp\ipykernel_5428\522615446.py:153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-

In [ ]:
import yfinance as yf
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from glob import glob

dfs = glob('df_fiis/df_fiis*')
lim = 0

# ==========================
# PARÂMETROS
# ==========================
PVP_MIN = 0.85
PVP_MAX = 1.00
LIQUIDEZ_MIN = 1.0      # R$ mi/dia
JANELA_QUEDA = 10
CDI = 0.15
SELIC_ANUAL = (1 + CDI) * (1 - 0.225)
PATRIMONIO_MIN = 500  # R$ 500 mi
NUM_COTISTAS_MIN = 10_000
DY_3M_MIN = 3
DY_6M_MIN = 6
DY_12M_MIN = 12

coluna_score = []
coluna_bloqueios = []
coluna_motivos = []
fiis_base = []

colunas_utilizadas = [
    'Fundos', 'Setor', 'Preço Atual (R$)', 'Liquidez Diária (milhões R$)',
    'P/VP', 'Último Dividendo', 'Dividend Yield', 'DY (3M) Acumulado',
    'DY (6M) Acumulado', 'DY (12M) Acumulado', 'DY Ano',
    'Patrimônio Líquido (milhões R$)', 'Quant. Ativos',
    'Num. Cotistas (milhares)'
]

df_fiis = df_fiis[colunas_utilizadas]

for i in df_fiis.Fundos.unique():
    try:
        ticker = yf.Ticker(f"{i}.SA")
        hist = ticker.history(period="1y")

        if len(hist) < 200:
            raise ValueError("Histórico insuficiente")

        retornos = hist["Close"].pct_change()

        # =====================================================
        # 🔥 CRITÉRIO DE VOLATILIDADE / EVENTO ATÍPICO (AJUSTADO)
        # =====================================================
        retorno_10d = (1 + retornos).rolling(JANELA_QUEDA).apply(
            lambda x: x.prod() - 1,
            raw=False
        )

        if pd.isna(retorno_10d.iloc[-1]):
            raise ValueError("Histórico insuficiente para janela de 10 dias")

        retorno_10d_atual = retorno_10d.iloc[-1]
        percentil_queda = (retorno_10d <= retorno_10d_atual).mean()

        # =====================================================
        score = 0
        bloqueios = []
        motivos = []

        # P/VP
        if PVP_MIN <= df_fiis[df_fiis["Fundos"] == i]["P/VP"].iloc[0] <= PVP_MAX:
            score += 1
            motivos.append("✅ Preço dentro da faixa saudável")
        else:
            bloqueios.append("❌ Preço fora da faixa saudável")

        # Liquidez
        if df_fiis[df_fiis["Fundos"] == i]["Liquidez Diária (milhões R$)"].iloc[0] >= LIQUIDEZ_MIN:
            score += 1
            motivos.append("✅ Liquidez adequada para entrada e saída")
        else:
            bloqueios.append("❌ Liquidez inadequada para operação segura")

        # Evento atípico de curto prazo
        if percentil_queda >= 0.15:
            score += 1
            motivos.append("✅ Queda recente compatível com o histórico do ativo")
        else:
            bloqueios.append("❌ Queda recente atípica vs histórico do ativo")

        # Patrimônio
        if df_fiis[df_fiis["Fundos"] == i]["Patrimônio Líquido (milhões R$)"].iloc[0] >= PATRIMONIO_MIN:
            score += 1
            motivos.append("✅ Escala patrimonial adequada")
        else:
            bloqueios.append("❌ Escala patrimonial abaixo do recomendado")

        # Cotistas
        num_cotistas = df_fiis[df_fiis["Fundos"] == i]["Num. Cotistas (milhares)"].iloc[0] * 1000
        if num_cotistas >= NUM_COTISTAS_MIN:
            score += 1
            if num_cotistas >= 150_000:
                motivos.append("✅ Base de cotistas consolidada")
            else:
                motivos.append("✅ Base de cotistas adequada")
        else:
            bloqueios.append("❌ Base de cotistas pouco representativa")

        # DY
        if (
            df_fiis[df_fiis["Fundos"] == i]["DY (3M) Acumulado"].iloc[0] >= DY_3M_MIN
            and df_fiis[df_fiis["Fundos"] == i]["DY (6M) Acumulado"].iloc[0] >= DY_6M_MIN
            and df_fiis[df_fiis["Fundos"] == i]["DY (12M) Acumulado"].iloc[0] >= DY_12M_MIN
        ):
            score += 1
            motivos.append("✅ Distribuição de rendimentos consistente")
        else:
            bloqueios.append("❌ Distribuição de rendimentos inconsistente")

        coluna_score.append(score)
        coluna_bloqueios.append(bloqueios)
        coluna_motivos.append(motivos)
        fiis_base.append(i)

    except Exception as e:
        coluna_score.append(0)
        coluna_bloqueios.append(["Informação insuficiente"])
        coluna_motivos.append([str(e)])
        fiis_base.append(i)

df_fiis = df_fiis[colunas_utilizadas]
df_fiis = df_fiis.drop_duplicates()
df_fiis['Motivos'] = coluna_motivos
df_fiis['Bloqueios'] = coluna_bloqueios
df_fiis['Score'] = coluna_score

df_fiis.to_parquet(f'df_fiis/df_fiis.parquet', index=False)
df_fiis.to_parquet(f'C:/Users/danie/OneDrive/Documentos/Refera_Django/refera_app/data/df_fiis.parquet', index=False)
df_fiis.to_parquet(f'df_fiis/df_fiis_{today_str}.parquet', index=False)


# df_top10.to_parquet(f'hist_top10/hist_top10_{today_str}.parquet', index=False)
# df_top10.to_parquet(f'hist_top10/df_top10.parquet', index=False)
# df_top10.to_parquet(f'C:/Users/danie/OneDrive/Documentos/Refera_Django/refera_app/data/df_top10.parquet', index=False)


$ARCT11.SA: possibly delisted; no price data found  (period=1y) (Yahoo error = "No data found, symbol may be delisted")
$ASMT11.SA: possibly delisted; no price data found  (period=1y) (Yahoo error = "No data found, symbol may be delisted")
$BBFI11B.SA: possibly delisted; no price data found  (period=1y) (Yahoo error = "No data found, symbol may be delisted")
$BICR11.SA: possibly delisted; no price data found  (period=1y) (Yahoo error = "No data found, symbol may be delisted")
$BLMC11.SA: possibly delisted; no price data found  (period=1y)
$BLMR11.SA: possibly delisted; no price data found  (period=1y)
$BPRP11.SA: possibly delisted; no price data found  (period=1y) (Yahoo error = "No data found, symbol may be delisted")
$BREV11.SA: possibly delisted; no price data found  (period=1y)
$BRIX11.SA: possibly delisted; no price data found  (period=1y) (Yahoo error = "No data found, symbol may be delisted")
$BRLA11.SA: possibly delisted; no price data found  (period=1y) (Yahoo error = "No data

In [ ]:
df_fiis.to_parquet(f'df_fiis/df_fiis.parquet', index=False)
df_fiis.to_parquet(f'C:/Users/danie/OneDrive/Documentos/Refera_Django/refera_app/data/df_fiis.parquet', index=False)
df_fiis.to_parquet(f'df_fiis/df_fiis_{today_str}.parquet', index=False)

# Construindo a base completa

In [ ]:
# x = pd.read_parquet('df_fiis/base_completa_df_fiis.parquet')
# x['data'] = '2025-12-21'
# x.to_parquet('df_fiis/base_completa_df_fiis.parquet')

In [5]:
from glob import glob

dfs = glob('df_fiis/df_fiis_*')
df_inicial = pd.read_parquet('df_fiis/base_completa_df_fiis.parquet')
datas_ja_inseridas = df_inicial.data.unique()

for i in dfs:
    data = i.split('.')[0].split('_')[-1]
    if data not in datas_ja_inseridas:
        print('Inserindo :',data)
        df_data = pd.read_parquet(i)
        df_data['data'] = data
        df_inicial = pd.concat([df_inicial,df_data])   
    else:
        print(data,'já inserida')

df_inicial.to_parquet('df_fiis/base_completa_df_fiis.parquet')
df_inicial.data.unique()


2025-12-21 já inserida
2025-12-22 já inserida
2025-12-29 já inserida
2025-12-30 já inserida
2026-01-03 já inserida
2026-01-04 já inserida


array(['2025-12-21', '2025-12-22', '2025-12-29', '2025-12-30',
       '2026-01-03', '2026-01-04'], dtype=object)

# Pegando os melhores e pegando os que não passam no critério

In [25]:
#lendo o arquivo mais atual 
from glob import glob
# dfs = glob('df_fiis/df_fiis_*')

# ultimo_arquivo = dfs[-1]
ultimo_arquivo = 'df_fiis/df_fiis.parquet'

base_atual = pd.read_parquet(ultimo_arquivo)

score_perfeitos = base_atual[base_atual.Score == 6]
score_bons = base_atual[(base_atual.Score >= 4) & (base_atual.Score < 6)]
score_obs = base_atual[(base_atual.Score == 3)]
score_ruins = base_atual[(base_atual.Score <= 2)]

print('Perfeitos :',score_perfeitos.shape)
print('Bons :',score_bons.shape)
print('Obs :',score_obs.shape)
print('Ruins :',score_ruins.shape)


Perfeitos : (24, 17)
Bons : (79, 17)
Obs : (67, 17)
Ruins : (288, 17)


In [26]:
score_perfeitos.value_counts('Score')

Score
6    24
Name: count, dtype: int64

In [29]:
score_perfeitos.Fundos.unique()

array(['BTCI11', 'BTHF11', 'CPSH11', 'CPTS11', 'GGRC11', 'HGCR11',
       'JSAF11', 'KNCA11', 'KNHF11', 'KNSC11', 'MCCI11', 'MCRE11',
       'RBRX11', 'RBRY11', 'RECR11', 'RURA11', 'RZAG11', 'RZAK11',
       'RZTR11', 'TGAR11', 'VCJR11', 'VGIR11', 'VRTA11', 'XPCI11'],
      dtype=object)

In [40]:
score_perfeitos.sort_values(['DY (12M) Acumulado'],ascending=False).sort_values(['DY (6M) Acumulado'],ascending=False).sort_values(['DY (3M) Acumulado'],ascending=False).sort_values('P/VP')

,Fundos,Setor,Preço Atual (R$),Liquidez Diária (milhões R$),P/VP,Último Dividendo,Dividend Yield,DY (3M) Acumulado,DY (6M) Acumulado,DY (12M) Acumulado,DY Ano,Patrimônio Líquido (milhões R$),Quant. Ativos,Num. Cotistas (milhares),Motivos,Bloqueios,Score
354,RURA11,Tijolo,8.58,2.212728,0.85,0.11,"1,25 %",3.93,7.95,15.47,"1,25 %",1649.960772,0,86.321,"[✅ Preço dentro da faixa saudável, ✅ Liquidez ...",[],6
329,RBRX11,Multiestratégia,8.37,3.315679,0.85,0.09,"1,11 %",3.43,7.04,13.94,"13,94 %",1430.587513,14,117.920,"[✅ Preço dentro da faixa saudável, ✅ Liquidez ...",[],6
210,JSAF11,Fundo de Fundos,7.71,2.297725,0.86,0.08,"1,03 %",3.28,6.89,14.82,"1,03 %",693.236380,14,23.208,"[✅ Preço dentro da faixa saudável, ✅ Liquidez ...",[],6
408,VCJR11,Papéis,79.50,1.853302,0.86,0.77,"0,95 %",3.27,6.72,15.42,"0,95 %",1387.626192,14,27.375,"[✅ Preço dentro da faixa saudável, ✅ Liquidez ...",[],6
394,TGAR11,Fundo de Desenvolvimento,91.21,5.169808,0.87,1.00,"1,08 %",3.41,7.12,14.61,"1,08 %",2533.636267,9,150.846,"[✅ Preço dentro da faixa saudável, ✅ Liquidez ...",[],6
82,CPTS11,Papéis,7.78,6.639158,0.87,0.09,"1,18 %",3.60,7.28,14.99,"14,99 %",2942.820945,14,352.554,"[✅ Preço dentro da faixa saudável, ✅ Liquidez ...",[],6
56,BTHF11,Papéis,8.89,4.782049,0.89,0.09,"1,09 %",3.25,6.60,14.51,"14,51 %",2066.545835,0,313.172,"[✅ Preço dentro da faixa saudável, ✅ Liquidez ...",[],6
139,GGRC11,Imóveis Industriais e Logísticos,9.93,9.940597,0.89,0.10,"1,01 %",3.02,6.14,12.74,"1,01 %",2396.843001,40,208.652,"[✅ Preço dentro da faixa saudável, ✅ Liquidez ...",[],6
340,RECR11,Papéis,82.45,2.958828,0.92,0.82,"1,03 %",3.02,6.44,15.61,"15,61 %",2367.515888,1,172.205,"[✅ Preço dentro da faixa saudável, ✅ Liquidez ...",[],6
255,MCRE11,Papéis,9.20,4.674807,0.92,0.11,"1,22 %",3.76,7.75,16.35,"16,35 %",1119.945715,1,86.943,"[✅ Preço dentro da faixa saudável, ✅ Liquidez ...",[],6


# Alerta Movimentação P/VP

In [53]:
base_completa = pd.read_parquet('df_fiis/base_completa_df_fiis.parquet')
# Preciso comparar um dia com o outro para ver os pvp
dfs = glob('df_fiis/df_fiis_*')
dia_anterior = dfs[-2].split('.')[0].split('_')[-1]
dia_hoje =  dfs[-1].split('.')[0].split('_')[-1]
print(dia_anterior)
print(dia_hoje)
# pegar o pvp dos fiis nesses dias 

print('Desceram para o critério')
for i in base_completa.Fundos.unique():
    pvp_anterior = base_completa[(base_completa.data == dia_anterior) & (base_completa.Fundos == i)]['P/VP'].iloc[0]
    pvp_atual = base_completa[(base_completa.data == dia_hoje) & (base_completa.Fundos == i)]['P/VP'].iloc[0]
    if pvp_atual < 1 and pvp_anterior >= 1:
        print(i, 'o pvp era', pvp_anterior,'e agora é ',pvp_atual)
print('________________________')
print('Subiram para o critério')
for i in base_completa.Fundos.unique():
    pvp_anterior = base_completa[(base_completa.data == dia_anterior) & (base_completa.Fundos == i)]['P/VP'].iloc[0]
    pvp_atual = base_completa[(base_completa.data == dia_hoje) & (base_completa.Fundos == i)]['P/VP'].iloc[0]
    if pvp_atual >= 0.85 and pvp_anterior < 0.85:
        print(i, 'o pvp era', pvp_anterior,'e agora é ',pvp_atual)




2026-01-03
2026-01-04
Desceram para o critério
AAZQ11 o pvp era 1.01 e agora é  0.99
BTYU11 o pvp era 1.0 e agora é  0.92
HGCR11 o pvp era 1.0 e agora é  0.99
HPDP11 o pvp era 1.0 e agora é  0.99
KNSC11 o pvp era 1.0 e agora é  0.99
WPLZ11 o pvp era 1.01 e agora é  0.93
________________________
Subiram para o critério
CXTL11 o pvp era 0.84 e agora é  0.87
FMOF11 o pvp era 0.76 e agora é  0.87


# Entrando no Score perfeito

In [59]:
base_completa = pd.read_parquet('df_fiis/base_completa_df_fiis.parquet')
# Preciso comparar um dia com o outro para ver os pvp
dfs = glob('df_fiis/df_fiis_*')
dia_anterior = dfs[-2].split('.')[0].split('_')[-1]
dia_hoje =  dfs[-1].split('.')[0].split('_')[-1]

# pegar o pvp dos fiis nesses dias 

for i in base_completa.Fundos.unique():
    score_anterior = base_completa[(base_completa.data == dia_anterior) & (base_completa.Fundos == i)]['Score'].iloc[0]
    score_atual = base_completa[(base_completa.data == dia_hoje) & (base_completa.Fundos == i)]['Score'].iloc[0]
    if score_atual >= 3 and score_anterior < 3:
        print(i,'passou a ter metade boa')

print('______')
for i in base_completa.Fundos.unique():
    score_anterior = base_completa[(base_completa.data == dia_anterior) & (base_completa.Fundos == i)]['Score'].iloc[0]
    score_atual = base_completa[(base_completa.data == dia_hoje) & (base_completa.Fundos == i)]['Score'].iloc[0]
    if score_atual < 3  and score_anterior >= 3:
        print(i,'caiu da metade')


FMOF11 passou a ter metade boa
HTMX11 passou a ter metade boa
______
RPRI11 caiu da metade


# Pegando Historicos dos preços

In [16]:
import yfinance as yf

ticker = yf.Ticker("XPML11.SA")
hist = ticker.history(period="12mo")  # ou "3mo", "6mo", "1y"
hist

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2025-01-02 00:00:00-03:00,88.575386,89.715807,87.102722,88.404770,166735,0.0,0.0
2025-01-03 00:00:00-03:00,88.404777,89.652950,87.066810,87.758247,160091,0.0,0.0
2025-01-06 00:00:00-03:00,88.476607,89.078243,87.758238,88.656204,126403,0.0,0.0
2025-01-07 00:00:00-03:00,88.656196,89.554163,88.162312,88.449661,111188,0.0,0.0
2025-01-08 00:00:00-03:00,88.674147,89.257827,87.102707,87.102707,126081,0.0,0.0
...,...,...,...,...,...,...,...
2025-12-23 00:00:00-03:00,108.410004,108.900002,108.139999,108.900002,139850,0.0,0.0
2025-12-26 00:00:00-03:00,109.000000,109.980003,108.180000,108.180000,153520,0.0,0.0
2025-12-29 00:00:00-03:00,109.269997,109.290001,107.839996,108.070000,131612,0.0,0.0


# Envio por whatsapp

In [35]:
import urllib.parse
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pyautogui

options = Options()
options.add_argument(r"--user-data-dir=C:\selenium\chrome_profile_whatsapp")
options.add_argument("--profile-directory=Default")

driver = webdriver.Chrome(options=options)

numero = "5513981832920"
mensagem = "Olá! Tudo bem? Teste FII."

lista_usuarios = {
    "5513981832920" : ['Daniel','CACR11', 'HGLG11'],
    "5527998976226" : ['Esther','XPML11', 'BTAL11']
}
time.sleep(10)
for numero in lista_usuarios.keys():
    mensagem = f'''Opa Boa noite! Tudo bem, {lista_usuarios[numero][0]}? Você pediu para eu validar os seguintes : {', '.join(lista_usuarios[numero][1:])}\n{lista_usuarios[numero][1]} : {df_fiis[df_fiis['Fundos'] == lista_usuarios[numero][1]]['Dividend Yield'].iloc[0]} \nQualquer dúvida estou à disposição!'''
    link = f"https://wa.me/{numero}?text={urllib.parse.quote(mensagem)}"
    print(link)
    # Abre o link do chat
    driver.get(link)

    wait = WebDriverWait(driver, 30)

    # Botão "Continuar para o chat"
    btn_continuar = wait.until(
        EC.element_to_be_clickable(
            (By.XPATH, "/html/body/div[1]/div[1]/div[2]/div/section/div/div/div/div[2]/div[4]/a[2]/span")
        )
    )
    btn_continuar.click()
    time.sleep(10)  # Espera o chat carregar
    # Pressiona ENTER (envia a mensagem)
    pyautogui.press("enter")
    time.sleep(1)
    pyautogui.hotkey('ctrl', 'w')

driver.quit()


https://wa.me/5513981832920?text=Opa%20Boa%20noite%21%20Tudo%20bem%2C%20Daniel%3F%20Voc%C3%AA%20pediu%20para%20eu%20validar%20os%20seguintes%20%3A%20CACR11%2C%20HGLG11%0ACACR11%20%3A%201%2C74%20%25%20%0AQualquer%20d%C3%BAvida%20estou%20%C3%A0%20disposi%C3%A7%C3%A3o%21
https://wa.me/5527998976226?text=Opa%20Boa%20noite%21%20Tudo%20bem%2C%20Esther%3F%20Voc%C3%AA%20pediu%20para%20eu%20validar%20os%20seguintes%20%3A%20XPML11%2C%20BTAL11%0AXPML11%20%3A%200%2C86%20%25%20%0AQualquer%20d%C3%BAvida%20estou%20%C3%A0%20disposi%C3%A7%C3%A3o%21
